In [1]:
import pandas as pd
import codecs, json
import os
import re

In [2]:
list_files = os.listdir('C:/Users/thgerner/Documents/twitter/example/singeldays/')

In [3]:
df_all_twitterjsons = pd.DataFrame(columns=['fullname', 'html', 'id', 'likes', 'replies', 'retweets', 'text', 'timestamp', 'url', 'user'])
for file in list_files:
    with codecs.open('C:/Users/thgerner/Documents/twitter/example/singeldays/'+file, 'r', 'utf-8') as f:
        tweets = json.load(f, encoding='utf-8')
    
    list_tweets = [list(elem.values()) for elem in tweets]
    list_columns = list(tweets[0].keys())
    df_dummy = pd.DataFrame(list_tweets, columns=list_columns)
    df_all_twitterjsons = pd.concat([df_all_twitterjsons,df_dummy],axis=0,join='outer',sort = False)


In [4]:
df_all_twitterjsons['date'] = pd.to_datetime(df_all_twitterjsons['timestamp'])

In [7]:
df_all_twitterjsons.columns
df_all_twitterjsons['text'][101]

'#Best #Can #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=216760691736418\xa0…'

In [6]:
df_all_twitterjsons = df_all_twitterjsons.drop(['html','id'],axis = 1)

In [7]:
df_all_twitterjsons['text'] = df_all_twitterjsons['text'].str.replace('\t',' ')
#df_all_twitterjsons['text'] = df_all_twitterjsons['text'].str.replace('\r','\n')
#df_all_twitterjsons['text'] = df_all_twitterjsons['text'].str.replace('\n\n','\n')
df_all_twitterjsons['text'] = df_all_twitterjsons['text'].str.replace('\n',' ')
df_all_twitterjsons['text'] = df_all_twitterjsons['text'].str.replace('\r',' ')

In [8]:
df_all_twitterjsons['url'] = df_all_twitterjsons['url'].apply(lambda x: 'https://twitter.com' + x)

In [9]:
#Add column with date yyyy-mm-dd
df_all_twitterjsons['date'] = df_all_twitterjsons['date'].dt.date

In [10]:
df_all_twitterjsons['lg'] = df_all_twitterjsons.apply(lambda row: 1 if any(x in row['text'].lower() for x in [' lg ',' #lg ', '#lgsmart','#lgrepairsupportinlubbocktexas','#lgrepairmaintenanceinwacotexas','#lgmicrowave','#lgmicrowaveoven','#lgrefrigerator']) else 0, axis=1)
df_all_twitterjsons['samsung'] = df_all_twitterjsons.apply(lambda row: 1 if any(x in row['text'].lower() for x in [' samsung ','#samsung','#samsungiot','#samsungrefrigerator','#samsungtest','#samsungsupport','#saynotosamsung']) else 0, axis=1)
df_all_twitterjsons['ge'] = df_all_twitterjsons.apply(lambda row: 1 if any(x in row['text'].lower() for x in [' ge ',' #ge ', ' general electric ', '#generalelectric','#gerefrigerator','#geprofile','#general #electric']) else 0, axis=1)
df_all_twitterjsons['whirlpool'] = df_all_twitterjsons.apply(lambda row: 1 if any(x in row['text'].lower() for x in [' whirlpool ','#whirlpool','#whirlpoolservicecenterdelhi','#whirlpoolservicecentergurgaon']) else 0, axis=1)
df_all_twitterjsons['mabe'] = df_all_twitterjsons.apply(lambda row: 1 if any(x in row['text'].lower() for x in [' mabe ',' #mabe ', '@mabe']) else 0, axis=1)
df_all_twitterjsons['haier'] = df_all_twitterjsons.apply(lambda row: 1 if any(x in row['text'].lower() for x in [' haier ',' #haier', ' #haieramerica ']) else 0, axis=1)

In [11]:
df_all_twitterjsons[['lg', 'samsung', 'ge', 'whirlpool','mabe']].sum()

lg           1799
samsung      5384
ge           1225
whirlpool    1920
mabe            6
dtype: int64

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    print(df_all_twitterjsons['text'][df_all_twitterjsons['ge'] == 1])

0       #Best #Ge #Counter #Depth #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=137412343037896 …                                                                                                                                                                                                                                                                                                     
5       #Best #Ge #Stainless #Steel #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=345495982134320 …                                                                                                                                                                                                                                                                                                   
35      #Best #Ge #Refrigerator #Wiring #Schematic #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=158429

In [37]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

In [93]:
from textblob import TextBlob 
#from textblob.sentiments import NaiveBayesAnalyzer

def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_tweet(tweet)) 
        return analysis.sentiment.polarity
        # set sentiment 
        #if analysis.sentiment.polarity > 0: 
        #    return 'positive'
        #elif analysis.sentiment.polarity == 0: 
        #    return 'neutral'
        #else: 
        #    return 'negative'

In [6]:
df_all_twitterjsons.reset_index(drop=True, inplace=True)

In [79]:
#For NaiveBayesAnalyzer: but NaiveBayesAnalyzer gives poor result(to many positive result, e.g. index = 20;3004)
#import nltk
#nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\thgerner\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

In [98]:
index = 20012
print(df_all_twitterjsons['text'][index])
print(clean_tweet(df_all_twitterjsons['text'][index]))
print(get_tweet_sentiment(df_all_twitterjsons['text'][index]))
#TextBlob(clean_tweet(df_all_twitterjsons['text'][index])

Lg LSSC243ST Studio... http://www.shortvale.com/9dd1z  #Counter #Depth #LG #LSSC243ST #Refrigerator #SeriesSideBySide #Studio #with
Lg LSSC243ST Studio Counter Depth LG LSSC243ST Refrigerator SeriesSideBySide Studio with
0.0


In [67]:
#TextBlob gives too many positive results, after a quick overview
df_all_twitterjsons['textblob_sentiment'] = df_all_twitterjsons.apply(lambda row: get_tweet_sentiment(row['text']), axis=1)

In [68]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

C:\Users\thgerner\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [69]:
sid = SentimentIntensityAnalyzer()
df_all_twitterjsons['total_sentiment'] = df_all_twitterjsons.apply(lambda row: sid.polarity_scores(row['text']), axis = 1)

In [70]:
compound = [d.get('compound') for d in df_all_twitterjsons.total_sentiment]
neg = [d.get('neg') for d in df_all_twitterjsons.total_sentiment]
neu = [d.get('neu') for d in df_all_twitterjsons.total_sentiment]
pos = [d.get('pos') for d in df_all_twitterjsons.total_sentiment]

df_all_twitterjsons['compound'] = compound
df_all_twitterjsons['neg'] = neg
df_all_twitterjsons['neu'] = neu
df_all_twitterjsons['pos'] = pos

In [99]:
df_all_twitterjsons['neg_compound'] = False
df_all_twitterjsons['neg_compound'][df_all_twitterjsons['compound'] < -0.5] = True
df_all_twitterjsons['neu_compound'] = False
df_all_twitterjsons['neu_compound'][(df_all_twitterjsons['compound'] > -0.5) & (df_all_twitterjsons['compound'] < 0.5)] = True
df_all_twitterjsons['pos_compound'] = False
df_all_twitterjsons['pos_compound'][df_all_twitterjsons['compound'] > 0.5] = True
df_all_twitterjsons['text_compound'] = 0
df_all_twitterjsons['text_compound'][df_all_twitterjsons['compound'] > 0.5] = 'Positive'
df_all_twitterjsons['text_compound'][df_all_twitterjsons['compound'] < -0.5] = 'Negative'
df_all_twitterjsons['text_compound'][(df_all_twitterjsons['compound'] > -0.5) & (df_all_twitterjsons['compound'] < 0.5)] = 'Neutral'

C:\Users\thgerner\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\thgerner\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\thgerner\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [103]:
df_all_twitterjsons.columns

Index(['fullname', 'likes', 'replies', 'retweets', 'text', 'timestamp', 'url',
       'user', 'date', 'lg', 'samsung', 'ge', 'whirlpool', 'mabe',
       'textblob_sentiment', 'total_sentiment', 'compound', 'neg', 'neu',
       'pos', 'neg_compound', 'neu_compound', 'pos_compound', 'text_compound'],
      dtype='object')

In [104]:
df_all_twitterjsons.to_csv('C:/Users/thgerner/Documents/twitter/example/powerbidata/twitter_historical.csv', index = False, sep = '\t', encoding = 'utf-8')

In [48]:
pd.set_option('display.max_colwidth', -1)
df_all_twitterjsons.iloc[0:14]

,fullname,likes,replies,retweets,text,timestamp,url,user
0,cepgubopmu83,0,0,0,#Best #Ge #Counter #Depth #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=137412343037896 …,2012-01-01T22:37:46,https://twitter.com/cepgubopmu83/status/153605862609518592,cepgubopmu83
1,Warehouse Appliance,0,0,0,Care for your #propane #refrigerator or #freezer: http://dld.bz/BT7T,2012-01-01T22:20:14,https://twitter.com/Warehouse_Aplnc/status/153601451782971393,Warehouse_Aplnc
2,birtimantu7,0,0,0,#Best #Refrigerator #Egg #Tray #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=138454106267053 …,2012-01-01T21:10:01,https://twitter.com/birtimantu7/status/153583780945342464,birtimantu7
3,cepgubopmu83,0,0,0,#Best #Clean #Steel #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=345460405470646 …,2012-01-01T19:28:49,https://twitter.com/cepgubopmu83/status/153558312581996546,cepgubopmu83
4,Reuben D. Springer,0,0,0,Amana : AFD2535DEW 25.0 cu. ft. Freestanding French Door #Refrigerator - White Review http://goo.gl/fb/SpvqD,2012-01-01T17:32:45,https://twitter.com/waterfilters26/status/153529103931289600,waterfilters26
5,cepgubopmu83,0,0,0,#Best #Ge #Stainless #Steel #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=345495982134320 …,2012-01-01T16:58:21,https://twitter.com/cepgubopmu83/status/153520446829113345,cepgubopmu83
6,Gil M. Lund,0,0,0,Electrolux : EW23BC71IS 36 22.6 cu. ft. Counter-Depth French-Door #Refrigerator - Stainless… http://goo.gl/fb/g25GS,2012-01-01T16:43:25,https://twitter.com/fridgefilters2/status/153516690544668672,fridgefilters2
7,hometoolkits,0,0,0,#Refrigerator #Frigidaire #Frigidaire Crosley Kenmore Electrolux Genuine http://www.hometoolkits.net/item/frigidaire-240342805meat-pan-for-refrigerator.html … Frigidaire 240342805Meat Pan for Refrigerator,2012-01-01T12:40:17,https://twitter.com/hometoolkits/status/153455501567475712,hometoolkits
8,birtimantu7,0,0,0,#Best #Amana #Refrigerator #Parts #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=201813229905831 …,2012-01-01T11:32:42,https://twitter.com/birtimantu7/status/153438495279562752,birtimantu7
9,Levi S Figueroa,0,0,0,#Samsung RF4287HARS 28 cu. ft. 4-Door French Door #Refrigerator - Stainless Steel Review http://goo.gl/fb/9cIUv,2012-01-01T11:19:15,https://twitter.com/faucetdepot1/status/153435108626542592,faucetdepot1


In [47]:
pd.set_option('display.max_colwidth', -1)
df_all_twitterjsons[df_all_twitterjsons['text'].str.contains('A #refrigerator that makes a')]

,fullname,likes,replies,retweets,text,timestamp,url,user
6,Appliance Washington,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:46,https://twitter.com/appwashington/status/334710711861248002,appwashington
7,appliance Silver,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:46,https://twitter.com/Appsilverspring/status/334710709055262720,Appsilverspring
8,appliancesterlingva,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:45,https://twitter.com/appliancesterli/status/334710707109122052,appliancesterli
9,Appliance Reston Va,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:45,https://twitter.com/Appliancereston/status/334710704655454208,Appliancereston
10,Appliance Woodbridge,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:44,https://twitter.com/Appwoodbridgeva/status/334710701966893056,Appwoodbridgeva
11,appliancepotomac,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:43,https://twitter.com/appliancepotoma/status/334710699077017601,appliancepotoma
12,Appliancemarlboro,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:43,https://twitter.com/Appliancemarlbo/status/334710695927095296,Appliancemarlbo
13,applianceherndon,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:42,https://twitter.com/applianceherndo/status/334710693305675777,applianceherndo
14,Appliance Bowie,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:42,https://twitter.com/ApplianceBowie/status/334710691086880769,ApplianceBowie
15,Appliance Alex. VA,0,0,0,A #refrigerator that makes a #buzzing or humming sound and doesn’t keep the #food cold may have a blown #compressor.,2013-05-15T16:43:41,https://twitter.com/ApplianceALEXVA/status/334710689367199744,ApplianceALEXVA


In [61]:
df_all_twitterjsons.head(1)

,fullname,likes,replies,retweets,text,timestamp,url,user,date
0,cepgubopmu83,0,0,0,#Best #Ge #Counter #Depth #Refrigerator #Deals #For #Sale #Free #Shipping http://www.facebook.com/note.php?note_id=137412343037896 …,2012-01-01T22:37:46,https://twitter.com/cepgubopmu83/status/153605862609518592,cepgubopmu83,2012-01-01


In [113]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    print(df_all_twitterjsons[(df_all_twitterjsons['user'].str.lower().contains('haier'))])

AttributeError: 'Series' object has no attribute 'contains'